In [2]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# train_err = pd.read_csv('./preprocessed data/new_train_err.csv', parse_dates=['time'])
train = pd.read_csv('./training set/train_case2.csv')
train_problem = pd.read_csv('./preprocessed data/new_train_problem.csv', parse_dates=['time'])
test = pd.read_csv('./training set/test.csv')
submission = pd.read_csv("C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/sample_submission.csv")

## quality columns 모두 제거

In [4]:
# train_quality
qual_col = list(train.iloc[:, -11:-3].columns)
train.drop(columns=qual_col, inplace=True)
train

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,fwver_03.11.1149,fwver_04.16.3439,fwver_04.16.3569,fwver_04.16.3571,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,fwver_04.22.1778,fwver_04.33.1125,fwver_04.33.1149,fwver_04.33.1171,fwver_04.33.1185,fwver_04.73.2571,fwver_04.82.1730,fwver_04.82.1778,fwver_05.15.2092,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.3104,fwver_05.66.3571,fwver_8.5.3,errtype_1,errtype_2,errtype_3,errtype_5,errtype_6,errtype_8,errtype_9,errtype_10,errtype_13,errtype_14,errtype_15,errtype_17,errtype_18,errtype_19,errtype_20,errtype_22,errtype_24,errtype_25,errtype_26,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period,multiple_model,multiple_fwver
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,-0.250725,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,0.085244,-0.296006,-0.075511,-0.052361,-0.037669,0.025586,0.015363,-0.026017,-0.652306,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.071956,-0.075181,-0.149787,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.442781,0,0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,1.267385,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,0.042137,-0.075511,-0.052361,-0.037669,-0.048693,0.015363,-0.034766,0.560808,-0.279724,4.832761,2.461897,5.213885,1.147012,0.066194,0.877285,-0.590355,-0.154519,0.339732,-0.198956,-0.166879,0.083164,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,0.442781,0,1
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,-0.250725,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.000473,-0.288962,-0.043544,-0.052361,-0.037669,-0.038082,0.015363,-0.052263,-0.753398,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.016696,-0.075181,-0.458185,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.272376,0,0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,0.079299,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.281917,-0.075511,-0.052361,-0.037669,-0.048693,-0.064734,-0.069761,-0.854491,-0.188357,-0.225162,-0.114028,-0.252677,-0.174717,-0.071956,-0.075181,0.951632,-0.154519,-0.194419,-0.198956,-0.343964,-0.061546,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,0.442781,0,1
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,0.164038,-0.01059,-0.019935,-0.012796,-0.250725,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.296006,-0.011578,-0.052361,-0.037669,-0.048693,0.175556,-0.047889,0.560808,-0.279724,-0.225162,-0.114028,-0.252677,0.028626,-0.071956,-0.075181,-0.105730,-0.154519,0.126072,-0.198956,0.364376,-0.061546,-0

In [5]:
# test_quality
qual_col = list(test.iloc[:, -11:-3].columns)
test.drop(columns=qual_col, inplace=True)
test

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,fwver_03.11.1149,fwver_04.16.3439,fwver_04.16.3569,fwver_04.16.3571,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,fwver_04.22.1778,fwver_04.33.1125,fwver_04.33.1149,fwver_04.33.1171,fwver_04.33.1185,fwver_04.73.2571,fwver_04.82.1730,fwver_04.82.1778,fwver_05.15.2092,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.3104,fwver_05.66.3571,fwver_8.5.3,errtype_1,errtype_2,errtype_3,errtype_5,errtype_6,errtype_8,errtype_9,errtype_10,errtype_13,errtype_14,errtype_15,errtype_17,errtype_18,errtype_19,errtype_20,errtype_22,errtype_24,errtype_25,errtype_26,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period,multiple_model,multiple_fwver
0,0.432273,-0.282718,1.465372,-0.138438,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,4.880865,-0.008226,-0.013014,-0.03182,-0.267318,-0.013322,-0.012829,-0.014172,-0.158979,-0.017038,-0.01172,-0.0421,-0.008166,-0.0083,-0.009566,-0.008195,-0.022131,-0.009703,-0.195694,-0.031262,-0.053793,0.023916,-0.076512,-0.047344,-0.056717,-0.022843,-0.144765,-0.045624,-0.563053,0.217326,2.989399,-0.08513,1.096973,1.435531,0.062686,2.924527,-0.142407,-0.153477,-0.056711,-0.194513,0.347687,-0.052885,0.017059,3.792035,-0.133970,0.040193,0.539686,-0.390797,-0.080517,0.269713,1.0,1.0
1,-0.191518,-0.282718,-0.227667,-0.190118,0.051198,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,-0.129828,-0.008226,-0.013014,-0.03182,-0.267318,-0.013322,-0.012829,-0.014172,-0.158979,-0.017038,-0.01172,-0.0421,-0.008166,-0.0083,-0.009566,-0.008195,-0.022131,-0.009703,-0.195694,-0.031262,0.051412,-0.228085,0.006710,-0.047344,-0.056717,-0.022843,0.676954,-0.036905,-0.752183,-0.121844,-0.202841,-0.08513,-0.190181,-0.212368,-0.081357,-0.115854,-0.495004,-0.153477,-0.139895,-0.199938,-1.000390,-0.052885,-0.075341,-1.005075,-0.133970,-0.086729,-0.602594,-0.390797,-0.645212,0.095698,0.0,0.0
2,-0.033722,0.638161,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,-0.129828,-0.008226,-0.013014,-0.03182,0.297034,-0.013322,-0.012829,-0.014172,-0.158979,-0.017038,-0.01172,-0.0421,-0.008166,-0.0083,-0.009566,-0.008195,-0.022131,-0.009703,-0.195694,-0.031262,-0.053793,0.005477,-0.048771,-0.047344,-0.056717,-0.022843,-0.144765,-0.051436,0.993630,-0.121844,-0.202841,-0.08513,-0.190181,-0.111784,-0.023740,-0.115854,0.210191,-0.153477,0.090616,-0.199938,0.347687,-0.052885,-0.075341,0.593961,-0.133970,0.801722,0.800423,-0.334183,-0.080517,0.443729,0.0,1.0
3,-0.180302,0.051777,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,-0.129828,-0.008226,-0.013014,-0.03182,0.092647,-0.013322,-0.012829,-0.014172,-0.158979,-0.017038,-0.01172,-0.0421,-0.008166,-0.0083,-0.009566,-0.008195,-0.022131,-0.009703,-0.195694,-0.031262,-0.053793,0.202160,-0.048771,-0.047344,-0.056717,-0.022843,0.060665,-0.033999,-1.406862,-0.121844,0.861239,-0.08513,1.096973,-0.207339,-0.081357,-0.115854,-1.332423,-0.153477,-0.005597,-0.199938,-0.663371,-0.004461,0.201860,0.593961,-0.133970,-0.086729,-0.105951,0.062114,-0.645212,0.095698,0.0,1.0
4,-0.041880,-0.282718,0.166889,0.017435,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,-0.129828,-0.008226,-0.013014,-0.03182,-0.267318,-0.013322,0.085206,-0.014172,-0.158979,-0.017038,-0.01172,-0.0421,-0.008166,-0.0083,-0.009566,-0.008195,-0.022131,-0.009703,-0.195694,-0.031262,-0.053793,-0.209646,-0.076512,-0.047344,-0.056717,-0.022843,0.060665,-0.025281,-0.373923,0.104270,-0.202841,-0.08513,-0.190181,-0.195604,-0.081357,-0.115854,-0.935751,-0.153477,0.192842,0.212372,-0.073587,-0.018296,0.017059,0.593961,0.288682

# 학습

In [6]:
train = train.to_numpy()
problem = np.zeros(15000)
problem[train_problem.user_id.unique() - 10000] = 1

train_x = train
train_y = problem

In [7]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):
    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 3633, number of negative: 8367
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4967
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302750 -> initscore=-0.834237
[LightGBM] [Info] Start training from score -0.834237
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.795184	valid_0's pr_auc: 0.801174
[LightGBM] [Info] Number of positive: 4828, number of negative: 7172
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025492 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5131
[LightGBM] [Info] Number of data points in the train set: 12000, number of used feature

In [8]:
print(np.mean(auc_scores))

0.798489815899112


In [9]:
train_x = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)
train_y.columns = ['problem']
train = pd.concat([train_x,train_y], axis =1)
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,problem
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,-0.250725,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,0.085244,-0.296006,-0.075511,-0.052361,-0.037669,0.025586,0.015363,-0.026017,-0.652306,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.071956,-0.075181,-0.149787,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.442781,0.0,0.0,0.0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,1.267385,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,0.042137,-0.075511,-0.052361,-0.037669,-0.048693,0.015363,-0.034766,0.560808,-0.279724,4.832761,2.461897,5.213885,1.147012,0.066194,0.877285,-0.590355,-0.154519,0.339732,-0.198956,-0.166879,0.083164,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,0.442781,0.0,1.0,1.0
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,-0.250725,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.000473,-0.288962,-0.043544,-0.052361,-0.037669,-0.038082,0.015363,-0.052263,-0.753398,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.016696,-0.075181,-0.458185,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.272376,0.0,0.0,0.0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,0.079299,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.281917,-0.075511,-0.052361,-0.037669,-0.048693,-0.064734,-0.069761,-0.854491,-0.188357,-0.225162,-0.114028,-0.252677,-0.174717,-0.071956,-0.075181,0.951632,-0.154519,-0.194419,-0.198956,-0.343964,-0.061546,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,0.442781,0.0,1.0,0.0
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,0.164038,-0.01059,-0.019935,-0.012796,-0.250725,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.296006,-0.011578,-0.052361,-0.037669,-0.048693,0.175556,-0.047889,0.560808,-0.279724,-0.225162,-0.114028,-0.252677,0.028626,-0.071956,-0.075181,-0.105730,-0.154519,0.126072,-0.198956,0.364376,-0.061546,-0.074159,0.628662,-0.114398,-0.080131,-0.552962,-0.336841,-0.080732,0.442781,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-0.316156,-0.340504,-0.239957,-0.053688,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,-0.250725,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.02633

In [10]:
clf = setup(data = train, target = "problem", silent=True, session_id=42) 

,Description,Value
0,session_id,42
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 67)"
5,Missing Values,False
6,Numeric Features,60
7,Categorical Features,6
8,Ordinal Features,False
9,High Cardinality Features,False


In [11]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7876,0.8147,0.4845,0.8129,0.6070,0.4740,0.5039,0.2820
rf,Random Forest Classifier,0.7884,0.8110,0.5025,0.7981,0.6166,0.4807,0.5053,0.1730
lightgbm,Light Gradient Boosting Machine,0.7894,0.8104,0.5250,0.7817,0.6280,0.4889,0.5079,0.0590
et,Extra Trees Classifier,0.7825,0.8067,0.4893,0.7885,0.6036,0.4649,0.4901,0.1660
ada,Ada Boost Classifier,0.7844,0.8049,0.4975,0.7882,0.6098,0.4711,0.4950,0.0800
lr,Logistic Regression,0.7749,0.7732,0.4227,0.8300,0.5599,0.4294,0.4737,0.3760
lda,Linear Discriminant Analysis,0.7609,0.7589,0.3605,0.8447,0.5051,0.3795,0.4394,0.0330
nb,Naive Bayes,0.7349,0.7370,0.3788,0.7125,0.4897,0.3333,0.3662,0.0150
knn,K Neighbors Classifier,0.7469,0.7311,0.4519,0.6945,0.5474,0.3825,0.3996,0.3460
dt,Decision Tree Classifier,0.6915,0.6607,0.5647,0.5433,0.5534,0.3180,0.3184,0.0310


In [12]:
blended = blend_models(estimator_list = best_3, optimize='AUC', method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7848,0.8053,0.4789,0.8057,0.6007,0.4661,0.4957
1,0.7857,0.8218,0.4901,0.7982,0.6073,0.4713,0.4978
2,0.7905,0.8025,0.5099,0.7974,0.6220,0.4866,0.5099
3,0.8010,0.8313,0.5365,0.8128,0.6464,0.5158,0.5374
4,0.7943,0.8065,0.4916,0.8333,0.6184,0.4901,0.5221
5,0.7867,0.7959,0.4972,0.7973,0.6125,0.4760,0.5012
6,0.7981,0.8243,0.5084,0.8303,0.6307,0.5026,0.5312
7,0.8010,0.8258,0.5197,0.8296,0.6390,0.5114,0.5381
8,0.7895,0.8307,0.5056,0.8000,0.6196,0.4842,0.5085


In [13]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7969,0.8071,0.4965,0.7931,0.6107,0.4830,0.5073


In [14]:
final_model = finalize_model(blended)

In [16]:
# pycaret의 predict_model을 사용하기 위해 test의 모든 컬럼을 정수로 변경
t_columns = list(np.arange(0,66))
test.columns = t_columns

In [17]:
predictions = predict_model(final_model, data=test, raw_score=True)

In [18]:
user_nan = predictions['Score_1.0'].mean()
new_row = pd.DataFrame(data=[user_nan], columns=['Score'])
new_row

,Score
0,0.43641


In [19]:
# 예측 결과값의 평균으로 넣어준다.
predictions = pd.concat([predictions[:13262], new_row, predictions[13262:]], ignore_index=True)
predictions.iloc[13262, :]

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
              ...   
65               NaN
Label            NaN
Score_0.0        NaN
Score_1.0        NaN
Score        0.43641
Name: 13262, Length: 70, dtype: object

In [20]:
submission['problem'] = predictions['Score_1.0']
submission

,user_id,problem
0,30000,0.9036
1,30001,0.2807
2,30002,0.4329
3,30003,0.7989
4,30004,0.8001
...,...,...
14994,44994,0.3507
14995,44995,0.4451
14996,44996,0.5951
14997,44997,0.8708


In [21]:
submission.to_csv('submission_case2_not_user_qual.csv', index=False)